In [1]:
!git clone https://github.com/selvaraj-sembulingam/ERA-V1.git
%cd ERA-V1/Assignments/S15

Cloning into 'ERA-V1'...
remote: Enumerating objects: 1899, done.
remote: Counting objects: 100% (797/797), done.
remote: Compressing objects: 100% (507/507), done.
remote: Total 1899 (delta 365), reused 642 (delta 282), pack-reused 1102
Receiving objects: 100% (1899/1899), 246.87 MiB | 18.75 MiB/s, done.
Resolving deltas: 100% (814/814), done.
/kaggle/working/ERA-V1/Assignments/S15


In [2]:
!pip install -q datasets tokenizers torchmetrics pytorch_lightning 

In [3]:
%%writefile datamodule.py
import pytorch_lightning as pl
from dataset import BilingualDataset

import torch
from torch.utils.data import DataLoader, random_split
from pathlib import Path

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

class OpusBookDataModule(pl.LightningDataModule):
  def __init__(self, config):
    super().__init__()
    self.config = config

  def get_all_sentences(self, ds, lang):
    for item in ds:
      yield item["translation"][lang]

  def get_or_build_tokenizer(self, ds, lang):
    tokenizer_path = Path(self.config["tokenizer_file"].format(lang))
    if not Path.exists(tokenizer_path):
      tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
      tokenizer.pre_tokenizer = Whitespace()
      trainer = WordLevelTrainer(
        special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2
      )
      tokenizer.train_from_iterator(self.get_all_sentences(ds, lang), trainer=trainer)
      tokenizer.save(str(tokenizer_path))
    else:
      tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer
    
  def setup(self, stage):
      # It only has the train split, so we divide it ourselves
      ds_raw = load_dataset(
        "opus_books", f"{self.config['lang_src']}-{self.config['lang_tgt']}", split="train"
      )

      # Build tokenizer
      self.tokenizer_src = self.get_or_build_tokenizer(ds_raw, self.config["lang_src"])
      self.tokenizer_tgt = self.get_or_build_tokenizer(ds_raw, self.config["lang_tgt"])

      # Keep 90% for training, 10% for validation
      train_ds_size = int(0.9 * len(ds_raw))
      val_ds_size = len(ds_raw) - train_ds_size
      train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

      self.train_ds = BilingualDataset(
        train_ds_raw,
        self.tokenizer_src,
        self.tokenizer_tgt,
        self.config["lang_src"],
        self.config["lang_tgt"],
        self.config["seq_len"],
      )
      self.val_ds = BilingualDataset(
        val_ds_raw,
        self.tokenizer_src,
        self.tokenizer_tgt,
        self.config["lang_src"],
        self.config["lang_tgt"],
        self.config["seq_len"],
      )

      # Find the maximum length of each sentence in the source and target sentence
      max_len_src = 0
      max_len_tgt = 0

      for item in ds_raw:
        src_ids = self.tokenizer_src.encode(item["translation"][self.config["lang_src"]]).ids
        tgt_ids = self.tokenizer_tgt.encode(item["translation"][self.config["lang_tgt"]]).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))

      print(f"Max length of source sentence: {max_len_src}")
      print(f"Max length of target sentence: {max_len_tgt}")

      return self.tokenizer_src, self.tokenizer_tgt



  def train_dataloader(self):     
      return DataLoader(self.train_ds, batch_size=self.config['batch_size'], shuffle=True)  
    
  def val_dataloader(self):
      return DataLoader(self.val_ds, batch_size=1, shuffle=True)

Writing datamodule.py


In [4]:
%%writefile litmodel.py
import torch
import torch.nn as nn
import torchmetrics
import pytorch_lightning as pl
import numpy 
from dataset import causal_mask

class LightningTransformer(pl.LightningModule):
    def __init__(self, model, learning_rate, tokenizer_src, tokenizer_tgt, max_len, num_examples):
        super(LightningTransformer, self).__init__()
        self.learning_rate = learning_rate
        self.model = model
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.loss_fn = nn.CrossEntropyLoss(ignore_index=self.tokenizer_src.token_to_id("[PAD]"), label_smoothing=0.1)
        self.max_len = max_len
        self.source_texts = []
        self.expected = []
        self.predicted = [] 
        self.num_examples = num_examples   
        self.cer_metric = torchmetrics.text.CharErrorRate()
        self.wer_metric = torchmetrics.text.WordErrorRate()
        self.bleu_metric = torchmetrics.text.BLEUScore()
        self.save_hyperparameters(ignore=['model'])   

    def training_step(self, batch, batch_idx):
        encoder_input = batch['encoder_input']
        decoder_input = batch['decoder_input']
        encoder_mask = batch['encoder_mask']
        decoder_mask = batch['decoder_mask']
        encoder_output = self.model.encode(encoder_input, encoder_mask)
        decoder_output = self.model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
        proj_output = self.model.project(decoder_output)
        label = batch['label']
        loss = self.loss_fn(proj_output.view(-1, self.tokenizer_tgt.get_vocab_size()), label.view(-1))

        self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=True)
        return loss

    def validation_step(self, batch, batch_idx):
        if batch_idx < self.num_examples:
            encoder_input = batch['encoder_input']
            encoder_mask = batch['encoder_mask']

            assert encoder_input.size(0) == 1, "Batch size must be 1 for validation"

            model_out = self.greedy_decode(encoder_input, encoder_mask)
            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = self.tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            self.source_texts.append(source_text)
            self.expected.append(target_text)
            self.predicted.append(model_out_text)

            # Print the source, target, and model output
            print("-"*10)
            print(f"{f'SOURCE: ':>12}{source_text}")
            print(f"{f'TARGET: ':>12}{target_text}")
            print(f"{f'PREDICTED: ':>12}{model_out_text}")

        if batch_idx == self.num_examples-1:
            cer = self.cer_metric(self.predicted, self.expected)
            self.log("val_cer", cer)

            wer = self.wer_metric(self.predicted, self.expected)
            self.log("val_wer", wer)   

            bleu = self.bleu_metric(self.predicted, self.expected) 
            self.log("val_bleu", bleu)

        if batch_idx >= self.num_examples:
            pass

    def greedy_decode(self, source, source_mask):
        sos_idx = self.tokenizer_tgt.token_to_id("[SOS]")
        eos_idx = self.tokenizer_tgt.token_to_id("[EOS]")

        # Precompute the encoder output and reuse it for every step
        encoder_output = self.model.encode(source, source_mask)
        # Initialize the decoder input with the sos token_to_id
        decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source)
        while True:
          if decoder_input.size(1) == self.max_len:
            break

          # build mask for target
          decoder_mask = (
            causal_mask(decoder_input.size(1)).type_as(source_mask)
          )

          # Calculate output
          out = self.model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

          # get next token_to_id
          prob = self.model.project(out[:, -1])
          _, next_word = torch.max(prob, dim=1)
          decoder_input = torch.cat(
            [
              decoder_input,
              torch.empty(1, 1).type_as(source).fill_(next_word.item()),
            ],
            dim=1,
          )

          if next_word == eos_idx:
            break
        return decoder_input.squeeze(0)

    def test_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, eps=1e-9)
        return optimizer

Writing litmodel.py


In [5]:
import torch
import warnings
import os
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning import Trainer, seed_everything

from datamodule import OpusBookDataModule
from litmodel import LightningTransformer
from config import get_config
cfg = get_config()


from model import build_transformer

if __name__ == '__main__':
  warnings.filterwarnings("ignore")
  torch.cuda.empty_cache()
  # os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:12240"

  pl.seed_everything(1, workers=True)
  data_module = OpusBookDataModule(cfg)
  tokenizer_src, tokenizer_tgt = data_module.setup(stage="None")
  model = build_transformer(
        tokenizer_src.get_vocab_size(), 
        tokenizer_tgt.get_vocab_size(),
        cfg["seq_len"],
        cfg["seq_len"],
        d_model=cfg["d_model"],
  )  
  
  litmodel = LightningTransformer(model=model, learning_rate=cfg["lr"], tokenizer_src=tokenizer_src, tokenizer_tgt=tokenizer_tgt, max_len=cfg["seq_len"], num_examples=2)


  trainer = pl.Trainer(
      max_epochs=10,
      deterministic=True,
      logger=True,
      enable_model_summary=False,
      log_every_n_steps=1,
      precision=16
  )
  trainer.fit(litmodel, data_module)



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

Dataset opus_books downloaded and prepared to /root/.cache/huggingface/datasets/opus_books/en-it/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf. Subsequent calls will reuse this data.
Max length of source sentence: 309
Max length of target sentence: 274
Max length of source sentence: 309
Max length of target sentence: 274


Sanity Checking: 0it [00:00, ?it/s]

----------
    SOURCE: Were you jealous, Jane?"
    TARGET: Eravate gelosa, Jane?
 PREDICTED: godono godono godono godono godono godono godono consigliò consigliò consigliò consigliò consigliò consigliò consigliò consigliò consigliò godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono consigliò circostante consigliò godono godono godono godono godono godono godono godono godono godono consigliò circostante consigliò godono godono godono godono godono godono godono godono godono consigliò circostante consigliò circostante consigliò circostante consigliò circostante consigliò circostante metà metà metà metà metà metà metà consigliò godono godono consigliò consigliò cranio metà metà metà metà metà godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono godono circostante circostante circostante circostante metà me

...

----------
    SOURCE: "Mr. Wood is in the vestry, sir, putting on his surplice."
    TARGET: — Il signor Wood è giunto e si veste.
 PREDICTED: — Il signor Rochester è in mezzo al sicuro , signore , e nel suo sangue .
----------
    SOURCE: 'WELL, HAVE YOU HAD A GOOD TIME?' she asked, coming out to meet him with a meek and repentant look on her face.
    TARGET: — Be’, c’è stata allegria? — chiese lei, uscendogli incontro con un’espressione colpevole e mansueta nel viso.
 PREDICTED: — Be ’, tu , sei stato fatto da te ? — ella disse , avvicinandosi a lui con un gesto energico , con un gesto spaventato e preoccupato .


Validation: 0it [00:00, ?it/s]

----------
    SOURCE: 'Yes, yes!'
    TARGET: — Sì, sì.
 PREDICTED: — Sì , sì !
----------
    SOURCE: What are you?
    TARGET: Cosa mai siete?
 PREDICTED: Che cosa avete ?


Validation: 0it [00:00, ?it/s]

----------
    SOURCE: [Long argument between Harris and Harris's friend as to what Harris is really singing.
    TARGET: (Lunga discussione fra Harris e l’amico di Harris su ciò che Harris realmente canti.
 PREDICTED: di Harris e di , come Harris di Harris , che è veramente veramente .
----------
    SOURCE: What an appetite I get in the country, wonderful!
    TARGET: Che appetito m’è venuto in campagna, un prodigio!
 PREDICTED: Che gioia ho fatto in campagna , in campagna !


In [8]:
model_filename = "transformer.pth"
torch.save(
            {
                "epoch": "10",
                "model_state_dict": model.state_dict(),
            },
            model_filename,
)